In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from itertools import product
from scipy import optimize # for fits

from sympy import latex, symbols, sin, cos, pi, simplify, lambdify, Matrix

%matplotlib inline
rng = np.random.RandomState(0)
from IPython.display import display, Math
import torch
import torch.autograd.functional as functional

In [2]:
def fgrad(inputs): #This function is useful to define something that the jocabian function will be able to differentiate.                 
    return torch.autograd.functional.jacobian(metric_tensor,inputs, create_graph=True) #Thus, we get an hessian at the end.
def christoffelderiv(coord1, coord2, deriv, alpha, beta): #This is the derivative of the Christoffel.
    christoffelderiv = 0.5*((jacobinv_l[alpha][beta][deriv] * (jacob_l[alpha][coord1][coord2] + jacob_l[alpha][coord2][coord1] - jacob_l[coord1][coord2][alpha]))\
                                    + g_inv_l[alpha][beta] * (hessian_l[alpha][coord1][coord2][deriv] + hessian_l[alpha][coord2][coord1][deriv] - hessian_l[coord1][coord2][alpha][deriv]))
    return christoffelderiv 


## Metric tensor: 2D unit sphere

Coordinates $x^\alpha = (\theta, \phi)$.

\begin{equation}
g_{\mu\nu} = 
\begin{pmatrix}
1 & 0 \\
0 & \sin^2 \theta \\
\end{pmatrix}
\end{equation}

In [6]:
dim=2 #The dimension of our metric tensor. 

def metric_tensor(coordinate,*arg):
    """
    Here, we have as arg for our function : 
    coordinates : (theta, phi) of a point 
    """
    theta=coordinate[0] #we define the values of our coordinate to reuse 
    phi=coordinate[1]
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1
    tensor[1,1] = torch.pow(torch.sin(theta),2)
    return tensor

def inverse(coordinate, *arg):
    theta=coordinate[0]
    phi=coordinate[1]
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1
    tensor[1,1] = torch.pow(torch.sin(theta),2)
    tensor = torch.inverse(tensor)
    return tensor


In [7]:
#Posons les coordonnées sur notre sphère : 
coord = torch.tensor([0.5, 1],requires_grad=True)
print("theta,phi coordinates: ",coord.data)

theta,phi coordinates:  tensor([0.5000, 1.0000])


## Metric tensor: 4D flat space, spherical coordinates

Coordinates $x^\alpha = (ct, R, \theta, \phi)$.

\begin{equation}
g_{\mu\nu} = 
\begin{pmatrix}
1 & 0 & 0 & 0  \\
0 & -1 & 0 & 0 \\
0 & 0 & -R^2 & 0 \\
0 & 0 & 0 & -R^2 \sin^2 \theta \\
\end{pmatrix}
\end{equation}

In [ ]:
dim=4

def metric_tensor(coordinate,*arg): 
    """
    Args: 
        coordinates (ct,r,theta,phi) of a point
        arg: not used
    """
    t=coordinate[0] # c=1
    r=coordinate[1]
    theta=coordinate[2]
    phi=coordinate[3]
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1
    tensor[1,1] = -1
    tensor[2,2] = -1 * torch.pow(r,2)
    tensor[3,3] = -1 * torch.pow(r*torch.sin(theta),2)
    return tensor

def inverse(coordinate, *arg):
    tensor = metric_tensor(coordinate, *arg)
    tensor = torch.inverse(tensor)
    return tensor

In [15]:
coord = torch.tensor([0, 1000, 30, 0.5],requires_grad=True)

## Metric tensor: Schwarzschid metric

Coordinates $x^\alpha = (ct, R, \theta, \phi)$.

$$g_{\mu\nu} = 
\begin{pmatrix}
1 - \frac{2GM}{Rc^2} & 0 & 0 & 0 \\
0 & -\left(1 - \frac{2GM}{Rc^2}\right)^{-1} & 0 & 0 \\
0 & 0 & -R^2 & 0 \\
0 & 0 & 0 & -R^2 \sin^2 \theta \\
\end{pmatrix}
= 
\begin{pmatrix}
1 - \frac{R_s}{R} & 0 & 0 & 0 \\
0 & -\left(1 - \frac{R_s}{R}\right)^{-1} & 0 & 0 \\
0 & 0 & -R^2 & 0 \\
0 & 0 & 0 & -R^2 \sin^2 \theta \\
\end{pmatrix}
$$

With $R_s = \frac{2 G M}{c^2}$, Schwarzschild radius.

In [3]:
dim=4
Rs = 1
def metric_tensor(coordinate,*arg):
    """
    Args: 
        coordinates (ct,r,theta,phi) of a point
        arg[0]: Rs, Schwarzschid radius
    """
    t=coordinate[0] # c=1
    r=coordinate[1]
    theta=coordinate[2]
    phi=coordinate[3]
    #if (len(arg)>0):
    #    Rs = arg[0] # Schwarzschid radius
    tensor = torch.zeros(dim,dim)
    tensor[0,0] = 1 - Rs/r
    tensor[1,1] = -1 * torch.pow(1 - Rs/r,-1)
    tensor[2,2] = -1 * torch.pow(r,2)
    tensor[3,3] = -1 * torch.pow(r*torch.sin(theta),2)
    return tensor

def inverse(coordinate, *arg):
    tensor = metric_tensor(coordinate, *arg)
    tensor = torch.inverse(tensor)
    return tensor

In [4]:
coord = torch.tensor([1,10,0.5,0.2],requires_grad=True)
print("t,r,theta,phi coordinates: ",coord.data)

t,r,theta,phi coordinates:  tensor([ 1.0000, 10.0000,  0.5000,  0.2000])


In [5]:
g = metric_tensor(coord)
g_l = g.tolist()
g_inv = torch.inverse(g)
g_inv_l = g_inv.tolist()
#We do all the preliminary calculations : 
jacob = functional.jacobian(metric_tensor,(coord)) #We take the jacobian of our metric tensor at the desirated coordinate
jacob_l = jacob.tolist()
jacobinv = functional.jacobian(inverse, (coord))
jacobinv_l = jacobinv.tolist()
hessian = torch.autograd.functional.jacobian(fgrad,(coord), create_graph=True)
hessian_l = hessian.tolist()

In [6]:
def Relat_calc():
    def christoffelproduct_liste(coord1, coord2, coord3):
        christoffelproduct = christoffel_l[beta][coord1][eta]*christoffel_l[eta][coord2][coord3]
        return christoffelproduct
    # Initialize Christoffel
    christoffel = torch.zeros(dim,dim,dim)
    for beta in range(0,dim):
        for mu in range(0,dim):
            for nu in range(0,dim):
                cricri = 0
                for alpha in range(0,dim):

                    cricri += 0.5 * g_inv_l[alpha][beta] * (jacob_l[alpha][mu][nu] + jacob_l[alpha][nu][mu] - jacob_l[mu][nu][alpha])
                christoffel[beta,mu,nu] = cricri
            
    christoffel_l = christoffel.tolist()
    # Initialize Riemannian tensor
    riemann = torch.zeros(dim,dim,dim,dim)
    riemann_l = riemann.tolist()
    for beta in range(0,dim):
        for epsilon in range(0,dim):
            for mu in range(0,dim):
                for nu in range(0,dim):
                    riemannscalar = 0
                    for alpha in range(0,dim):
                        riemannscalar += christoffelderiv(mu, epsilon, nu, alpha, beta)-christoffelderiv(mu, nu, epsilon, alpha, beta)
                    for eta in range(0,dim):
                        riemannscalar += christoffelproduct_liste(nu, mu, epsilon) - christoffelproduct_liste(epsilon, mu, nu) 
                    
                    riemann_l[beta][mu][nu][epsilon] = riemannscalar

    #Initialize Ricci tensor
    ricci = torch.zeros(dim,dim)
    ricci_l = ricci.tolist()
    for mu in range(0, dim):
        for nu in range(0, dim):
            riccisca = 0
            for alpha in range(0, dim):
                riccisca += riemann_l[alpha][mu][alpha][nu]
                ricci_l[mu][nu]= riccisca
    #scalarcurvature 
    scalarcurvature = 0
    for mu in range(dim):
        for nu in range(dim):
            scalarcurvature += g_inv_l[mu][nu]*ricci_l[mu][nu]
    i = 0
    while i < 1:
        print("What do you want to display ? Choose a number : \n 1 : Christoffel_symbols \n 2 : Riemann_tensor \n 3 : Ricci_tensor \n 4 : Scalar_curvature ")
        detail = input()
        if detail== '1':
            for beta in range(0,dim):
                for mu in range(0,dim):
                    for nu in range(0,dim):
                        display(Math(f'\Gamma^{{{beta}}}_{{{mu} {nu}}} = ' + latex(christoffel_l[beta][mu][nu])))
        elif detail=='2':
            for beta in range(0,dim):
                for epsilon in range(0,dim):
                    for mu in range(0,dim):
                        for nu in range(0,dim):
                            display(Math(f'R^{{{beta}}}_{{{mu} {nu} {epsilon}}} = ' + latex(riemann_l[beta][mu][nu][epsilon])))
        elif detail=='3':
            for mu in range(0, dim):
                for nu in range(0, dim):
                    display(Math(f'R_{{{mu} {nu}}} = ' + latex(ricci_l[mu][nu])))
        elif detail=='4':
            display(Math(f'R = ' + latex(scalarcurvature)))
        else :
            print("Error : please choose a number between 1, 2, 3 and 4.")
        print("Do you want to display an other value ? y/n")
        x = input()
        if x == 'n':
            i += 1
Relat_calc()


What do you want to display ? Choose a number : 
 1 : Christoffel_symbols 
 2 : Riemann_tensor 
 3 : Ricci_tensor 
 4 : Scalar_curvature 
1


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Do you want to display an other value ? y/n
y
What do you want to display ? Choose a number : 
 1 : Christoffel_symbols 
 2 : Riemann_tensor 
 3 : Ricci_tensor 
 4 : Scalar_curvature 
4


<IPython.core.display.Math object>

Do you want to display an other value ? y/n
n
